In [1]:
from med_associates_utils.session import parse_directory, SessionCollection
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
import pandas as pd

mpl.rcParams['pdf.fonttype'] = 42

In [2]:
base_path = r'C:\Users\thackray\git\med-associates-utils\sample_data\CRF MedAssoc Files\CRF MedAssoc Files'
data_sources = {
    os.path.join(base_path, "MT - Day 1"): {'Genotype': 'MT', 'Day': 'Day1'},
    os.path.join(base_path, "MT - Day 2"): {'Genotype': 'MT', 'Day': 'Day2'},
    os.path.join(base_path, "MT - Day 3"): {'Genotype': 'MT', 'Day': 'Day3'},
    os.path.join(base_path, "MT - Day 4"): {'Genotype': 'MT', 'Day': 'Day4'},

    os.path.join(base_path, "WT - Day 1"): {'Genotype': 'WT', 'Day': 'Day1'},
    os.path.join(base_path, "WT - Day 2"): {'Genotype': 'WT', 'Day': 'Day2'},
    os.path.join(base_path, "WT - Day 3"): {'Genotype': 'WT', 'Day': 'Day3'},
    os.path.join(base_path, "WT - Day 4"): {'Genotype': 'WT', 'Day': 'Day4'},
}

sessions = SessionCollection()
for path, meta in data_sources.items():
    print(f'reading path "{path}"')
    ss = parse_directory(path)
    ss.update_metadata(meta)
    sessions.extend(ss)

def set_animal_id(session):
    animal_id = re.search("EN#\d+", session.metadata['Subject']).group(0)
    session.metadata['Animal'] = animal_id

sessions.apply(set_animal_id)

reading path "C:\Users\thackray\git\med-associates-utils\sample_data\CRF MedAssoc Files\CRF MedAssoc Files\MT - Day 1"


100%|██████████| 30/30 [00:00<00:00, 2068.67it/s]


reading path "C:\Users\thackray\git\med-associates-utils\sample_data\CRF MedAssoc Files\CRF MedAssoc Files\MT - Day 2"


100%|██████████| 30/30 [00:00<00:00, 1764.46it/s]


reading path "C:\Users\thackray\git\med-associates-utils\sample_data\CRF MedAssoc Files\CRF MedAssoc Files\MT - Day 3"


...

100%|██████████| 26/26 [00:00<00:00, 2260.52it/s]


reading path "C:\Users\thackray\git\med-associates-utils\sample_data\CRF MedAssoc Files\CRF MedAssoc Files\WT - Day 3"


100%|██████████| 26/26 [00:00<00:00, 1999.52it/s]


reading path "C:\Users\thackray\git\med-associates-utils\sample_data\CRF MedAssoc Files\CRF MedAssoc Files\WT - Day 4"


100%|██████████| 26/26 [00:00<00:00, 2079.52it/s]


In [3]:
sessions.metadata

,StartDate,EndDate,Subject,Experiment,Group,Box,StartTime,StartDateTime,EndTime,MSN,Genotype,Day,Animal
0,2022-05-08,2022-05-08,1_EN#20_1215,20220508,1,4,07:16:07,2022-05-08 07:16:07,08:46:25,CRF,MT,Day1,EN#20
1,2022-05-08,2022-05-08,EN#26_1016,20220508,7,4,17:41:51,2022-05-08 17:41:51,19:11:59,CRF,MT,Day1,EN#26
2,2022-05-08,2022-05-08,EN#76_0924,20220508,7,1,17:41:21,2022-05-08 17:41:21,19:11:54,CRF,MT,Day1,EN#76
3,2022-05-08,2022-05-08,EN#77_0924,20220508,7,2,17:41:27,2022-05-08 17:41:27,19:11:56,CRF,MT,Day1,EN#77
4,2022-05-13,2022-05-13,EN#35_1101,20220513,8,2,12:58:24,2022-05-13 12:58:24,13:54:32,CRF,MT,Day1,EN#35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2023-03-04,2023-03-04,2_EN#50-1125,20230304,2,2,10:05:08,2023-03-04 10:05:08,10:31:15,CRFv2023,WT,Day4,EN#50
220,2023-03-04,2023-03-04,4_EN#33-1012,20230304,4,4,11:10:05,2023-03-04 11:10:05,12:34:00,CRFv2023,WT,Day4,EN#33
221,2023-03-04,2023-03-04,4_EN#44-1126,20230304,4,3,11:39:38,2023-03-04 11:39:38,11:54:45,CRFv2023,WT,Day4,EN#44
222,2023-03-04,2023-03-04,4_EN#31-1012,20230304,4,1,11:45:14,2023-03-04 11:45:14,12:44:01,CRFv2023,WT,Day4,EN#31


In [4]:
sessions.describe()

Number of sessions: 224

Arrays present in data with counts:
(224) "A"
(224) "B"
(224) "C"
(224) "D"
(60) "F"

Scalars present in data with counts:
(224) "R"
(224) "S"




In [5]:
def rename_arrays_by_program(session):
    if session.metadata['MSN'] in ['CRF', 'CRF-Cue2-rewire']:
        session.rename_array('B', 'rewarded_nosepoke')
        session.rename_array('C', 'magazine_entry')
        session.rename_array('D', 'reward_retrieval_latency')

    elif session.metadata['MSN'] == 'CRFv2023':
        session.rename_array('B', 'nosepoke')
        session.rename_array('C', 'magazine_entry')
        session.rename_array('D', 'reward_retrieval_latency')
        session.rename_array('F', 'rewarded_nosepoke')

sessions.apply(rename_arrays_by_program)
sessions.describe()

Number of sessions: 224

Arrays present in data with counts:
(224) "A"
(224) "rewarded_nosepoke"
(224) "magazine_entry"
(224) "reward_retrieval_latency"
(60) "nosepoke"

Scalars present in data with counts:
(224) "R"
(224) "S"




In [6]:
event_df = sessions.to_dataframe(include_meta=['Genotype', 'Day', 'Animal', 'Subject'])
event_df

,StartDate,EndDate,Subject,Experiment,Group,Box,StartTime,StartDateTime,EndTime,MSN,Genotype,Day,Animal,event,time
9280,2022-11-06,2022-11-06,7_EN#59-0709,20221106,7,1,13:30:33,2022-11-06 13:30:33,13:46:46,CRF,MT,Day4,EN#59,reward_retrieval_latency,1.39
9893,2023-03-04,2023-03-04,3_EN#61-0101,20230304,3,1,11:23:22,2023-03-04 11:23:22,11:39:53,CRFv2023,MT,Day4,EN#61,reward_retrieval_latency,1.44
17277,2022-11-06,2022-11-06,6_EN#60-0823,20221106,6,2,11:45:51,2022-11-06 11:45:51,11:58:23,CRF,WT,Day4,EN#60,reward_retrieval_latency,1.45
7660,2022-05-16,2022-05-16,EN#26_1220,20220516,9,3,16:39:19,2022-05-16 16:39:19,16:57:17,CRF,MT,Day4,EN#26,reward_retrieval_latency,1.46
7661,2022-05-16,2022-05-16,EN#26_1220,20220516,9,3,16:39:19,2022-05-16 16:39:19,16:57:17,CRF,MT,Day4,EN#26,reward_retrieval_latency,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15737,2023-03-03,2023-03-03,4_EN#31-1012,20230303,4,1,10:03:48,2023-03-03 10:03:48,11:33:56,CRFv2023,WT,Day3,EN#31,nosepoke,5385.06
789,2022-11-03,2022-11-03,2_EN#81-0824,20221103,2,3,08:17:06,2022-11-03 08:17:06,09:47:09,CRF,MT,Day1,EN#81,rewarded_nosepoke,5390.22
12257,2022-05-14,2022-05-14,EN#49_1224,20220514,10,3,15:43:35,2022-05-14 15:43:35,17:13:39,CRF,WT,Day2,EN#49,rewarded_nosepoke,5392.14
790,2022-11-03,2022-11-03,2_EN#81-0824,20221103,2,3,08:17:06,2022-11-03 08:17:06,09:47:09,CRF,MT,Day1,EN#81,magazine_entry,5396.94
